# Big Data Project

## Configuration

In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [82]:
//Denys: val bucketname = "unibo-bd2122-dgrushchak"
//Riccardo: val bucketname = "unibo-bd2223-rbacca"

val bucketname = "unibo-bd2122-dgrushchak"

//Denys: val path_flights_db = "s3a://"+bucketname+"/project/small_file_10000000.txt"
//Riccardo:val path_flights_db = "s3a://"+bucketname+"/bigdata-project/part_1.txt"

val path_flights_db = "s3a://"+bucketname+"/projectTest/xaa.txt"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2122-dgrushchak
path_flights_db: String = s3a://unibo-bd2122-dgrushchak/projectTest/xaa.txt
res172: String = application_1674580030687_0002
res174: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1674580030687_0002/


In [86]:
object Parser{
    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val doubleVerticalLine = "\\|\\|"
    
    val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)" // not used
    val quotes = "\"" //not used
   
    
    case class FlightData(
        flightDate: String,          //1
        startingAirport: String,     //2
        destinationAirport: String,  //3
        travelDuration: Long,        //4
        airplaneType: Array[String], //5
        airplaneName: Array[String], //6
        isNonStop: Boolean,          //7
        isBasicEconomy: Boolean,     //8
        totalFare: Double            //9
    ){
        def un() = FlightData.unapply(this).get
    }
    
    def convertTravelDuration(value: String): Long = {
        val input = value.replace("PT", "")
        val minutes = input.replace("M", "").split("H") //[2, 35]
        
        (minutes(0).trim.toLong)*60 + minutes(1).trim.toLong
    }
    
    def convertAirplaneType(value: String): Array[String] = {
        //Ex. Airbus A320||Boeing 737-800
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    def convertAirlineName(value: String): Array[String] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
     /**
        2 - flightdate (String) ** x._1
        3 - starting airport (String) ** x._2
        4 - destination airport (String) ** x._3
        6 - travel duration (PT2H35M) ** in minutes x._4
        8 - isBasicEconomy (String) ** x._8
        9 - isrefundable (boolean)
        10 - isNonStop (String) ** x._7
        12 - basefare (prezzo base del ticket in $ Double) ** x._9
        12 - seats remaining (Int)
        20 - airline name (company name) ** x._6
        23 - airplane type (Vector of [String||String] ** x._5
        24 - segment distancd (distance in milles: Vector of [Int||Int]  can be [None||None])
    **/
     def parseFlightInformationLine(line: String): Option[FlightData] = {
        try {
            val input = line.split(commaRegex)
            
            val airplanes = convertAirplaneType(input(23))
            val airline = convertAirlineName(input(21))

            if (airplanes.isEmpty || airline.isEmpty) None
            else Some(FlightData(input(2), input(3), input(4), convertTravelDuration(input(6)), airplanes, airline, input(10).toBoolean, input(8).toBoolean, input(12).toDouble))
        } catch {
            case _: Exception => None
        }
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object Parser


In [87]:
sc.getPersistentRDDs.foreach(_._2.unpersist()) 
val rddFlights = sc.textFile(path_flights_db).flatMap(Parser.parseFlightInformationLine)
val rddFlightsCached = rddFlights.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFlights: org.apache.spark.rdd.RDD[Parser.FlightData] = MapPartitionsRDD[181] at flatMap at <console>:25
rddFlightsCached: rddFlights.type = MapPartitionsRDD[181] at flatMap at <console>:25


In [88]:
rddFlightsCached.map(_.un()).take(10).foreach(println)


def round(v: Double): Double = {
    (v*100).toInt/100.toDouble
}

val numberOfRecords = rddFlightsCached.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2022-04-17,ATL,BOS,149,[Ljava.lang.String;@5f9d370c,[Ljava.lang.String;@5deedb99,true,false,248.6)
(2022-04-17,ATL,BOS,150,[Ljava.lang.String;@4a3a3180,[Ljava.lang.String;@54eb17c2,true,false,248.6)
(2022-04-17,ATL,BOS,150,[Ljava.lang.String;@62787f04,[Ljava.lang.String;@496152c1,true,false,248.6)
(2022-04-17,ATL,BOS,152,[Ljava.lang.String;@52ccdc5c,[Ljava.lang.String;@5e1584c0,true,false,248.6)
(2022-04-17,ATL,BOS,154,[Ljava.lang.String;@7f344059,[Ljava.lang.String;@3c80e224,true,false,248.6)
(2022-04-17,ATL,BOS,252,[Ljava.lang.String;@89af7d4,[Ljava.lang.String;@7adb4183,false,false,251.1)
(2022-04-17,ATL,BOS,318,[Ljava.lang.String;@2730f009,[Ljava.lang.String;@1c9ab709,false,false,251.1)
(2022-04-17,ATL,BOS,332,[Ljava.lang.String;@4361d719,[Ljava.lang.String;@631651af,false,false,251.1)
(2022-04-17,ATL,BOS,398,[Ljava.lang.String;@4386457c,[Ljava.lang.String;@aed80fa,false,false,251.1)
(2022-04-17,ATL,BOS,286,[Ljava.lang.String;@477957a1,[Ljava.lang.String;@16b3fdc8,false,false,252.

## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline, only in non-stop flights
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights


In [89]:
//1.How many distinct airports and aircraft models
val distinctAirports = rddFlightsCached.
    map(x => x.startingAirport).
    distinct.union(
        rddFlightsCached.map(x => x.destinationAirport).distinct).
    distinct.count
val distinctAircraftModels = rddFlightsCached.flatMap(x => x.airplaneType).distinct.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

distinctAirports: Long = 16
distinctAircraftModels: Long = 43


In [90]:
//2. Average travel duration for each airline, in non stop flights
val distinctAirlines = rddFlightsCached.
    filter(x => x.isNonStop).
    map(x => (x.airplaneName(0), x.travelDuration)).
    aggregateByKey((0.0, 0.0))((a,v)=>(a._1+v, a._2+1), (a1,a2)=>(a1._1+a2._1, a1._2+a2._2)).
    map({case(k,v)=>(k,v._1/v._2)}).
    collect().
    foreach({case (airline, value) => println(airline + " => " + round(value) + " avg minutes")})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Frontier Airlines => 165.84 avg minutes
United => 186.45 avg minutes
Delta => 152.53 avg minutes
Spirit Airlines => 178.69 avg minutes
Alaska Airlines => 243.72 avg minutes
American Airlines => 183.45 avg minutes
JetBlue Airways => 200.58 avg minutes
distinctAirlines: Unit = ()


In [91]:
//3. Percentage of basic economy tickets, based on all tickets
round((rddFlightsCached.filter(_.isBasicEconomy).count.toDouble/numberOfRecords).toDouble*100) + " %"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res183: String = 8.34 %


In [92]:
//4. Percentage of non-stop flights (flights with one leg)
round((rddFlightsCached.filter(_.isNonStop).count.toDouble/numberOfRecords).toDouble*100) + " %"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res185: String = 28.09 %


In [93]:
//5. Average and price range of tickets
val ticketPrices = rddFlightsCached.map(x => x.totalFare) // TODO with all dataset try performance with and without a cache
"Range of prices: "  + ticketPrices.min + " to " + ticketPrices.max
"Avg price: " + round((ticketPrices.sum/numberOfRecords).toDouble)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ticketPrices: org.apache.spark.rdd.RDD[Double] = MapPartitionsRDD[205] at map at <console>:24
res187: String = Range of prices: 23.97 to 3810.6
res188: String = Avg price: 351.64


In [94]:
//6. Average ticket price for each airline
val avgTicketPricePerAirline = rddFlightsCached.
    filter(_.isNonStop).
    map(x => (x.airplaneName(0),(x.totalFare, 1))).
    reduceByKey((a,b) => (a._1+b._1, a._2+b._2)).
    map(m => (m._1,m._2._1/m._2._2)).
    collect.foreach({case (name, value) => println(name + " => " + round(value))})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Frontier Airlines => 112.7
United => 283.57
Delta => 249.71
Spirit Airlines => 133.39
Alaska Airlines => 277.86
American Airlines => 267.75
JetBlue Airways => 231.55
avgTicketPricePerAirline: Unit = ()


In [ ]:
//7. Average and range of travel distance

In [95]:
//8. Top 10 airports with more arriving flights
val topAirports = rddFlightsCached.
    map(x => (x.destinationAirport, 1)).
    reduceByKey(_+_).
    sortBy(_._2, false).
    take(10).
    foreach({case (name, value) => println(name + " => " + value)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LAX => 43019
LGA => 36384
DFW => 32115
MIA => 32009
BOS => 31642
ORD => 31537
CLT => 30388
ATL => 30196
SFO => 30048
DEN => 28267
topAirports: Unit = ()


## Personal works

> Basta una query compicata a testa

Query approfondite:

- Denys:
    - Aggrego “airline” e “aircraft model” per determinare il prezzo medio di un miglio nautico, e dopo aggrego (MIN) su “aircraft model” per trovare l'airline che vende i biglietti più economici per quel modello.
    - Aggrego su “aircraft model” per calcolare la “travel distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.
- Riccardo:
    - Aggrego su “aircraft models” per calcolare la classifica discreta dei modelli più usati rispetto a ciascuna “airline”. Eseguo il join col dataset originale, infine riaggrego su “travel duration” e sulla classificazione di prima. Ottengo la durata di media di ogni volo per ogni compagnia (oltre a numero di voli e totale di ore) sul modello di aereo più utilizzato
    - Aggrego su giorno ed airline per calcolare la distanza totale, poi aggrego su airline per calcolare la media dei totali giornalieri (di distanza) per ciascuna compagnia aerea
    - Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni copia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo